# beginner

In [ ]:
!nvidia-smi

Mon Jun 10 07:15:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# %load_ext cudf.pandas

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [ ]:
df.drop(labels=["id", "clean_title"], axis=1, inplace=True)
df_test.drop(labels=["id", "clean_title"], axis=1, inplace=True)

In [ ]:
df.shape

(54273, 11)

In [ ]:
df.isna().sum()

brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
price           0
dtype: int64

## transforming-columns

In [ ]:
columns_to_transform_mean = ['brand', 'model', 'engine', 'transmission', 'ext_col', 'int_col']

In [ ]:
for col in columns_to_transform_mean:
    mean_encoder = df.groupby(col)['price'].mean()
    df[f'{col}_encoded'] = df[col].map(mean_encoder)
    df_test[f'{col}_encoded'] = df_test[col].map(mean_encoder)

In [ ]:
df.drop(labels=columns_to_transform_mean, axis=1, inplace=True)
df_test.drop(labels=columns_to_transform_mean, axis=1, inplace=True)

In [ ]:
df.head()

,model_year,milage,fuel_type,accident,price,brand_encoded,model_encoded,engine_encoded,transmission_encoded,ext_col_encoded,int_col_encoded
0,2018,74349,Gasoline,None reported,11000,38154.063227,43169.955645,53064.682749,60508.125743,37614.687018,26723.437460
1,2007,80000,Gasoline,None reported,8250,40276.029448,15520.375546,23719.608815,33931.317579,38413.911328,41290.661314
2,2009,91491,Gasoline,None reported,15000,34840.403933,14397.047619,16442.580952,24939.541386,26112.148936,27862.756473
3,2022,2437,Hybrid,None reported,63500,40276.029448,68657.677778,60728.788546,45993.777037,45155.865090,49465.203662
4,2001,111000,Gasoline,None reported,7850,17526.060403,10793.000000,27903.848214,29971.027033,37774.187602,41290.661314


In [ ]:
df["fuel_type"].unique()

array(['Gasoline', 'Hybrid', 'E85 Flex Fuel', 'Diesel', '–',
       'Plug-In Hybrid', 'not supported'], dtype=object)

In [ ]:
df.groupby('fuel_type')['price'].mean()

fuel_type
Diesel            52801.151488
E85 Flex Fuel     24788.251521
Gasoline          38940.499747
Hybrid            51727.819366
Plug-In Hybrid    43514.280220
not supported     13098.750000
–                 29869.333333
Name: price, dtype: float64

In [ ]:
fuel_encoder = {
    'Gasoline':4,
    'Hybrid':2,
    'E85 Flex Fuel':6,
    'Diesel':1,
    '–':5,
    'Plug-In Hybrid':3,
    'not supported':7,
}

In [ ]:
df["accident"].unique()

array(['None reported', 'At least 1 accident or damage reported'],
      dtype=object)

In [ ]:
df.groupby('accident')['price'].mean()

accident
At least 1 accident or damage reported    25302.986645
None reported                             44233.044315
Name: price, dtype: float64

In [ ]:
accident_encoder = {
    'None reported':1,
    'At least 1 accident or damage reported':2,
}

In [ ]:
df['fuel_type_encoded'] = df['fuel_type'].map(fuel_encoder)
df['accident_encoded'] = df['accident'].map(accident_encoder)
df_test['fuel_type_encoded'] = df_test['fuel_type'].map(fuel_encoder)
df_test['accident_encoded'] = df_test['accident'].map(accident_encoder)

In [ ]:
df.drop(labels=['accident','fuel_type'], axis=1, inplace=True)
df_test.drop(labels=['accident','fuel_type'], axis=1, inplace=True)

In [ ]:
y = np.log1p(df['price'])
df.drop(labels=['price'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [ ]:
x = scaler.fit_transform(df)
df_test = df_test.fillna(df_test.mean())
x_test = scaler.transform(df_test)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33)

In [ ]:
# Importing required libraries
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,BatchNormalization,Dropout
from keras.layers import LeakyReLU,Conv2D,MaxPooling2D,AveragePooling2D
from keras import regularizers
import time

In [ ]:
model = Sequential()
model.add(Dense(units=10))

model.add(Dense(units=16))

model.add(Dense(units=32))
model.add(LeakyReLU())

model.add(Dense(units=64))
model.add(BatchNormalization())

model.add(Dense(units=128))
model.add(LeakyReLU())

model.add(Dense(units=256))
model.add(BatchNormalization())
model.add(Dense(units=256))
model.add(LeakyReLU())
model.add(Dense(units=128))
model.add(LeakyReLU())
model.add(Dense(units=256))
model.add(BatchNormalization())
model.add(Dense(units=256))
model.add(LeakyReLU())
model.add(Dropout(0.1))
model.add(Dense(units=512))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(Dense(units=512))
model.add(LeakyReLU())
model.add(Dense(units=128))
model.add(Dropout(0.1))
model.add(Dense(units=256))
model.add(BatchNormalization())
model.add(Dense(units=256))
model.add(LeakyReLU())
model.add(Dense(units=128))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(units=64))
model.add(LeakyReLU())
model.add(Dense(units=32))
model.add(Dense(units=16))
model.add(Dense(units=1))

# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dropout(0.1))

In [ ]:
# Defining Loss_function,Optimizer,Metrics and compiling the model
loss_function = keras.losses.MeanSquaredError()
OPTIMIZER = keras.optimizers.SGD(learning_rate=0.0001)
METRICS = [keras.metrics.RootMeanSquaredError()]

model.compile(
    loss=loss_function,
    optimizer=OPTIMIZER,
    metrics=METRICS
)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
# Saving the model with callback
CKPT_path = os.path.join("Models","Model_ckpt.h5")
checkpoint_cb = keras.callbacks.ModelCheckpoint(CKPT_path, save_best_only=True)

In [ ]:
EPOCHS = 200 # Number of Epochs
VALIDATION_SET = (x_val,y_val) # Validation data
history = model.fit(x_train,y_train,epochs=EPOCHS,validation_data=VALIDATION_SET,batch_size=128,
                    callbacks=[early_stopping_cb,checkpoint_cb],use_multiprocessing=True)

Epoch 1/200
285/285 [==============================] - 12s 14ms/step - loss: 18.1777 - root_mean_squared_error: 4.2635 - val_loss: 4.0386 - val_root_mean_squared_error: 2.0096
Epoch 2/200
285/285 [==============================] - 4s 13ms/step - loss: 1.8776 - root_mean_squared_error: 1.3703 - val_loss: 0.7614 - val_root_mean_squared_error: 0.8726
Epoch 3/200
285/285 [==============================] - 5s 16ms/step - loss: 1.5156 - root_mean_squared_error: 1.2311 - val_loss: 0.5931 - val_root_mean_squared_error: 0.7701
Epoch 4/200
285/285 [==============================] - 4s 14ms/step - loss: 1.3322 - root_mean_squared_error: 1.1542 - val_loss: 0.5549 - val_root_mean_squared_error: 0.7449
Epoch 5/200
285/285 [==============================] - 3s 12ms/step - loss: 1.2188 - root_mean_squared_error: 1.1040 - val_loss: 0.5601 - val_root_mean_squared_error: 0.7484
Epoch 6/200
285/285 [==============================] - 4s 13ms/step - loss: 1.1388 - root_mean_squared_error: 1.0671 - val_loss:

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(history.history).plot() # Plotting all of the accuracy and losses

In [ ]:
x_test.shape

In [ ]:
ckpt_model = keras.models.load_model(CKPT_path) # Loading model
y_pred_log = ckpt_model.predict(x_test)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor

In [ ]:
reg1 = GradientBoostingRegressor(n_estimators=400)
reg2 = GradientBoostingRegressor(n_estimators=200)
reg3 = RandomForestRegressor(n_estimators=400)
reg4 = RandomForestRegressor(n_estimators=200)
reg5 = HistGradientBoostingRegressor(max_iter=400)
reg6 = HistGradientBoostingRegressor(max_iter=200)
reg7 = AdaBoostRegressor(n_estimators=400)
reg8 = AdaBoostRegressor(n_estimators=200)
reg9 = ExtraTreesRegressor(n_estimators=400)
reg10 = ExtraTreesRegressor(n_estimators=200)

In [ ]:
vreg = VotingRegressor([
    ('reg1',reg1),
    ('reg2',reg2),
    ('reg3',reg3),
    ('reg4',reg4),
    ('reg5',reg5),
    ('reg6',reg6),
    ('reg7',reg7),
    ('reg8',reg8),
    ('reg9',reg9),
    ('reg10',reg10),
], verbose=True)
vreg.fit(x_train, y_train)

[Voting] .................... (1 of 10) Processing reg1, total=  17.6s
[Voting] .................... (2 of 10) Processing reg2, total=   7.5s
[Voting] .................... (3 of 10) Processing reg3, total= 1.2min
[Voting] .................... (4 of 10) Processing reg4, total=  35.4s
[Voting] .................... (5 of 10) Processing reg5, total=   0.5s
[Voting] .................... (6 of 10) Processing reg6, total=   0.5s
[Voting] .................... (7 of 10) Processing reg7, total=   0.8s
[Voting] .................... (8 of 10) Processing reg8, total=   0.7s
[Voting] .................... (9 of 10) Processing reg9, total=  36.9s
[Voting] .................. (10 of 10) Processing reg10, total=  18.1s


VotingRegressor(estimators=[('reg1',
                             GradientBoostingRegressor(n_estimators=400)),
                            ('reg2',
                             GradientBoostingRegressor(n_estimators=200)),
                            ('reg3', RandomForestRegressor(n_estimators=400)),
                            ('reg4', RandomForestRegressor(n_estimators=200)),
                            ('reg5',
                             HistGradientBoostingRegressor(max_iter=400)),
                            ('reg6',
                             HistGradientBoostingRegressor(max_iter=200)),
                            ('reg7', AdaBoostRegressor(n_estimators=400)),
                            ('reg8', AdaBoostRegressor(n_estimators=200)),
                            ('reg9', ExtraTreesRegressor(n_estimators=400)),
                            ('reg10', ExtraTreesRegressor(n_estimators=200))],
                verbose=True)

In [ ]:
y_pred_log = vreg.predict(x_test)

In [ ]:
y_pred = np.ravel(np.expm1(y_pred_log))

In [ ]:
y_pred

array([22173.0304595 , 17741.68172148, 31398.47133902, ...,
        9530.14456924, 49649.70557617, 13846.52559893])

In [ ]:
test_ids = np.array(pd.read_csv("test.csv")["id"])

In [ ]:
test_ids

array([54273, 54274, 54275, ..., 90453, 90454, 90455])

In [ ]:
submission = pd.DataFrame({
    'id': test_ids,
    'price': y_pred
})

In [ ]:
submission.isna().sum()

id       0
price    0
dtype: int64

In [ ]:
submission.to_csv('submission.csv', index=False)

# catbooster

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
print(df_train.shape)
print(df_test.shape)

(54273, 13)
(36183, 12)


In [ ]:
df_train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,Yes,11000
1,1,BMW,335 i,2007,80000,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Black,None reported,Yes,8250
2,2,Jaguar,XF Luxury,2009,91491,Gasoline,300.0HP 4.2L 8 Cylinder Engine Gasoline Fuel,6-Speed A/T,Purple,Beige,None reported,Yes,15000
3,3,BMW,X7 xDrive40i,2022,2437,Hybrid,335.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,Transmission w/Dual Shift Mode,Gray,Brown,None reported,Yes,63500
4,4,Pontiac,Firebird Base,2001,111000,Gasoline,200.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,A/T,White,Black,None reported,Yes,7850


In [ ]:
df_train.drop(labels=['id','clean_title'], axis=1, inplace=True)
df_test.drop(labels=['id','clean_title'], axis=1, inplace=True)

In [ ]:
y_train = df_train['price']
x_train = df_train.drop(labels=['price'], axis=1)

In [ ]:
cat_features = ['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident']

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

In [ ]:
reg1 = GradientBoostingRegressor(n_estimators=200)
reg2 = GradientBoostingRegressor(n_estimators=150)
reg3 = RandomForestRegressor(n_estimators=200)
reg4 = RandomForestRegressor(n_estimators=150)
reg5 = HistGradientBoostingRegressor(max_iter=200)
reg6 = HistGradientBoostingRegressor(max_iter=150)
reg7 = AdaBoostRegressor(n_estimators=200)
reg8 = AdaBoostRegressor(n_estimators=150)
reg9 = ExtraTreesRegressor(n_estimators=200)
reg10 = ExtraTreesRegressor(n_estimators=150)
# reg11 = CatBoostRegressor(iterations=1000)
# reg12 = CatBoostRegressor(iterations=1200)

In [ ]:
vreg = VotingRegressor([
    ('reg1',reg1),
    ('reg2',reg2),
    ('reg3',reg3),
    ('reg4',reg4),
    ('reg5',reg5),
    ('reg6',reg6),
    ('reg7',reg7),
    ('reg8',reg8),
    ('reg9',reg9),
    ('reg10',reg10),
], verbose=True)
vreg.fit(x_train, y_train)

ValueError: could not convert string to float: 'Ford'

In [ ]:
reg.fit(x_train, y_train, cat_features)

0:	learn: 72593.9032511	total: 116ms	remaining: 9m 40s
1:	learn: 72395.5398265	total: 205ms	remaining: 8m 32s
2:	learn: 72164.9358833	total: 278ms	remaining: 7m 42s
3:	learn: 71975.5596044	total: 381ms	remaining: 7m 55s
4:	learn: 71764.9771810	total: 465ms	remaining: 7m 44s
5:	learn: 71568.2682465	total: 545ms	remaining: 7m 33s
6:	learn: 71401.9554621	total: 641ms	remaining: 7m 37s
7:	learn: 71212.4328106	total: 710ms	remaining: 7m 23s
8:	learn: 71043.1374003	total: 803ms	remaining: 7m 25s
9:	learn: 70858.2589839	total: 893ms	remaining: 7m 25s
10:	learn: 70732.0298190	total: 975ms	remaining: 7m 22s
11:	learn: 70569.4126709	total: 1.06s	remaining: 7m 20s
12:	learn: 70393.3538700	total: 1.15s	remaining: 7m 22s
13:	learn: 70103.6109422	total: 1.24s	remaining: 7m 20s
14:	learn: 69880.6315365	total: 1.33s	remaining: 7m 22s
15:	learn: 69714.8636696	total: 1.44s	remaining: 7m 28s
16:	learn: 69511.0468648	total: 1.52s	remaining: 7m 24s
17:	learn: 69369.6783482	total: 1.6s	remaining: 7m 22s
18:

In [ ]:
y_pred = reg.predict(df_test)

In [ ]:
test_ids = np.array(pd.read_csv("test.csv")["id"])

In [ ]:
submission = pd.DataFrame({
    'id': test_ids,
    'price': y_pred
})

In [ ]:
submission.shape    # 36183

(36183, 2)

In [ ]:
submission.isna().sum()

id       0
price    0
dtype: int64

In [ ]:
submission.to_csv('submission.csv', index=False)